REF: https://github.com/FIAP/genetic_algorithm_tsp

In [ ]:
import pygame
from pygame.locals import *
import random
import itertools
# from genetic_algorithm import mutate, order_crossover, generate_random_population, calculate_fitness, sort_population, default_problems
# from draw_functions import draw_paths, draw_plot, draw_cities
import sys
import numpy as np
# from benchmark_att48 import *

In [ ]:
# FUNÇÕES DO PROFESSOR
# genetic_algorithm.py

import random
import math
import copy
from typing import List, Tuple

default_problems = {
5: [(733, 251), (706, 87), (546, 97), (562, 49), (576, 253)],
10:[(470, 169), (602, 202), (754, 239), (476, 233), (468, 301), (522, 29), (597, 171), (487, 325), (746, 232), (558, 136)],
12:[(728, 67), (560, 160), (602, 312), (712, 148), (535, 340), (720, 354), (568, 300), (629, 260), (539, 46), (634, 343), (491, 135), (768, 161)],
15:[(512, 317), (741, 72), (552, 50), (772, 346), (637, 12), (589, 131), (732, 165), (605, 15), (730, 38), (576, 216), (589, 381), (711, 387), (563, 228), (494, 22), (787, 288)]
}

def generate_random_population(cities_location: List[Tuple[float, float]], population_size: int) -> List[List[Tuple[float, float]]]:
    """
    Generate a random population of routes for a given set of cities.

    Parameters:
    - cities_location (List[Tuple[float, float]]): A list of tuples representing the locations of cities,
      where each tuple contains the latitude and longitude.
    - population_size (int): The size of the population, i.e., the number of routes to generate.

    Returns:
    List[List[Tuple[float, float]]]: A list of routes, where each route is represented as a list of city locations.
    """
    return [random.sample(cities_location, len(cities_location)) for _ in range(population_size)]


def calculate_distance(point1: Tuple[float, float], point2: Tuple[float, float]) -> float:
    """
    Calculate the Euclidean distance between two points.

    Parameters:
    - point1 (Tuple[float, float]): The coordinates of the first point.
    - point2 (Tuple[float, float]): The coordinates of the second point.

    Returns:
    float: The Euclidean distance between the two points.
    """
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)


def calculate_fitness(path: List[Tuple[float, float]]) -> float:
    """
    Calculate the fitness of a given path based on the total Euclidean distance.

    Parameters:
    - path (List[Tuple[float, float]]): A list of tuples representing the path,
      where each tuple contains the coordinates of a point.

    Returns:
    float: The total Euclidean distance of the path.
    """
    distance = 0
    n = len(path)
    for i in range(n):
        distance += calculate_distance(path[i], path[(i + 1) % n])

    return distance


def order_crossover(parent1: List[Tuple[float, float]], parent2: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
    """
    Perform order crossover (OX) between two parent sequences to create a child sequence.

    Parameters:
    - parent1 (List[Tuple[float, float]]): The first parent sequence.
    - parent2 (List[Tuple[float, float]]): The second parent sequence.

    Returns:
    List[Tuple[float, float]]: The child sequence resulting from the order crossover.
    """
    length = len(parent1)

    # Choose two random indices for the crossover
    start_index = random.randint(0, length - 1)
    end_index = random.randint(start_index + 1, length)

    # Initialize the child with a copy of the substring from parent1
    child = parent1[start_index:end_index]

    # Fill in the remaining positions with genes from parent2
    remaining_positions = [i for i in range(length) if i < start_index or i >= end_index]
    remaining_genes = [gene for gene in parent2 if gene not in child]

    for position, gene in zip(remaining_positions, remaining_genes):
        child.insert(position, gene)

    return child

### demonstration: crossover test code
# Example usage:
# parent1 = [(1, 1), (2, 2), (3, 3), (4,4), (5,5), (6, 6)]
# parent2 = [(6, 6), (5, 5), (4, 4), (3, 3),  (2, 2), (1, 1)]

# # parent1 = [1, 2, 3, 4, 5, 6]
# # parent2 = [6, 5, 4, 3, 2, 1]


# child = order_crossover(parent1, parent2)
# print("Parent 1:", [0, 1, 2, 3, 4, 5, 6, 7, 8])
# print("Parent 1:", parent1)
# print("Parent 2:", parent2)
# print("Child   :", child)


# # Example usage:
# population = generate_random_population(5, 10)

# print(calculate_fitness(population[0]))


# population = [(random.randint(0, 100), random.randint(0, 100))
#           for _ in range(3)]



# TODO: implement a mutation_intensity and invert pieces of code instead of just swamping two.
def mutate(solution:  List[Tuple[float, float]], mutation_probability: float) ->  List[Tuple[float, float]]:
    """
    Mutate a solution by inverting a segment of the sequence with a given mutation probability.

    Parameters:
    - solution (List[int]): The solution sequence to be mutated.
    - mutation_probability (float): The probability of mutation for each individual in the solution.

    Returns:
    List[int]: The mutated solution sequence.
    """
    mutated_solution = copy.deepcopy(solution)

    # Check if mutation should occur
    if random.random() < mutation_probability:

        # Ensure there are at least two cities to perform a swap
        if len(solution) < 2:
            return solution

        # Select a random index (excluding the last index) for swapping
        index = random.randint(0, len(solution) - 2)

        # Swap the cities at the selected index and the next index
        mutated_solution[index], mutated_solution[index + 1] = solution[index + 1], solution[index]

    return mutated_solution

### Demonstration: mutation test code
# # Example usage:
# original_solution = [(1, 1), (2, 2), (3, 3), (4, 4)]
# mutation_probability = 1

# mutated_solution = mutate(original_solution, mutation_probability)
# print("Original Solution:", original_solution)
# print("Mutated Solution:", mutated_solution)


def sort_population(population: List[List[Tuple[float, float]]], fitness: List[float]) -> Tuple[List[List[Tuple[float, float]]], List[float]]:
    """
    Sort a population based on fitness values.

    Parameters:
    - population (List[List[Tuple[float, float]]]): The population of solutions, where each solution is represented as a list.
    - fitness (List[float]): The corresponding fitness values for each solution in the population.

    Returns:
    Tuple[List[List[Tuple[float, float]]], List[float]]: A tuple containing the sorted population and corresponding sorted fitness values.
    """
    # Combine lists into pairs
    combined_lists = list(zip(population, fitness))

    # Sort based on the values of the fitness list
    sorted_combined_lists = sorted(combined_lists, key=lambda x: x[1])

    # Separate the sorted pairs back into individual lists
    sorted_population, sorted_fitness = zip(*sorted_combined_lists)

    return sorted_population, sorted_fitness


if __name__ == '__main__':
    N_CITIES = 10

    POPULATION_SIZE = 100
    N_GENERATIONS = 100
    MUTATION_PROBABILITY = 0.3
    cities_locations = [(random.randint(0, 100), random.randint(0, 100))
              for _ in range(N_CITIES)]

    # CREATE INITIAL POPULATION
    population = generate_random_population(cities_locations, POPULATION_SIZE)

    # Lists to store best fitness and generation for plotting
    best_fitness_values = []
    best_solutions = []

    for generation in range(N_GENERATIONS):


        population_fitness = [calculate_fitness(individual) for individual in population]

        population, population_fitness = sort_population(population,  population_fitness)

        best_fitness = calculate_fitness(population[0])
        best_solution = population[0]

        best_fitness_values.append(best_fitness)
        best_solutions.append(best_solution)

        print(f"Generation {generation}: Best fitness = {best_fitness}")

        new_population = [population[0]]  # Keep the best individual: ELITISM

        while len(new_population) < POPULATION_SIZE:

            # SELECTION
            parent1, parent2 = random.choices(population[:10], k=2)  # Select parents from the top 10 individuals

            # CROSSOVER
            child1 = order_crossover(parent1, parent2)

            ## MUTATION
            child1 = mutate(child1, MUTATION_PROBABILITY)

            new_population.append(child1)


        print('generation: ', generation)
        population = new_population

Generation 0: Best fitness = 360.5111426483346
generation:  0
Generation 1: Best fitness = 286.05860676236523
generation:  1
Generation 2: Best fitness = 286.05860676236523
generation:  2
Generation 3: Best fitness = 286.05860676236523
generation:  3
Generation 4: Best fitness = 273.6965230972647
generation:  4
Generation 5: Best fitness = 273.6965230972647
generation:  5
Generation 6: Best fitness = 273.6965230972647
generation:  6
Generation 7: Best fitness = 273.6965230972647
generation:  7
Generation 8: Best fitness = 273.6965230972647
generation:  8
Generation 9: Best fitness = 273.6965230972647
generation:  9
Generation 10: Best fitness = 273.6965230972647
generation:  10
Generation 11: Best fitness = 273.6965230972647
generation:  11
Generation 12: Best fitness = 273.6965230972647
generation:  12
Generation 13: Best fitness = 273.6965230972647
generation:  13
Generation 14: Best fitness = 273.6965230972647
generation:  14
Generation 15: Best fitness = 273.6965230972647
generatio

In [ ]:
# FUNÇÕES DO PROFESSOR
# draw_functions.py

# -*- coding: utf-8 -*-
"""
Created on Fri Dec 22 16:03:11 2023

@author: SérgioPolimante
"""
import pylab
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib
import pygame
from typing import List, Tuple

matplotlib.use("Agg")


def draw_plot(screen: pygame.Surface, x: list, y: list, x_label: str = 'Generation', y_label: str = 'Fitness') -> None:
    """
    Draw a plot on a Pygame screen using Matplotlib.

    Parameters:
    - screen (pygame.Surface): The Pygame surface to draw the plot on.
    - x (list): The x-axis values.
    - y (list): The y-axis values.
    - x_label (str): Label for the x-axis (default is 'Generation').
    - y_label (str): Label for the y-axis (default is 'Fitness').
    """
    fig, ax = plt.subplots(figsize=(4, 4), dpi=100)
    ax.plot(x, y)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    plt.tight_layout()

    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    renderer = canvas.get_renderer()
    raw_data = renderer.tostring_rgb()

    size = canvas.get_width_height()
    surf = pygame.image.fromstring(raw_data, size, "RGB")
    screen.blit(surf, (0, 0))

def draw_cities(screen: pygame.Surface, cities_locations: List[Tuple[int, int]], rgb_color: Tuple[int, int, int], node_radius: int) -> None:
    """
    Draws circles representing cities on the given Pygame screen.

    Parameters:
    - screen (pygame.Surface): The Pygame surface on which to draw the cities.
    - cities_locations (List[Tuple[int, int]]): List of (x, y) coordinates representing the locations of cities.
    - rgb_color (Tuple[int, int, int]): Tuple of three integers (R, G, B) representing the color of the city circles.
    - node_radius (int): The radius of the city circles.

    Returns:
    None
    """
    for city_location in cities_locations:
        pygame.draw.circle(screen, rgb_color, city_location, node_radius)



def draw_paths(screen: pygame.Surface, path: List[Tuple[int, int]], rgb_color: Tuple[int, int, int], width: int = 1):
    """
    Draw a path on a Pygame screen.

    Parameters:
    - screen (pygame.Surface): The Pygame surface to draw the path on.
    - path (List[Tuple[int, int]]): List of tuples representing the coordinates of the path.
    - rgb_color (Tuple[int, int, int]): RGB values for the color of the path.
    - width (int): Width of the path lines (default is 1).
    """
    pygame.draw.lines(screen, rgb_color, True, path, width=width)


def draw_text(screen: pygame.Surface, text: str, color: pygame.Color) -> None:
    """
    Draw text on a Pygame screen.

    Parameters:
    - screen (pygame.Surface): The Pygame surface to draw the text on.
    - text (str): The text to be displayed.
    - color (pygame.Color): The color of the text.
    """
    pygame.font.init()  # You have to call this at the start

    font_size = 15
    my_font = pygame.font.SysFont('Arial', font_size)
    text_surface = my_font.render(text, False, color)

    cities_locations = []  # Assuming you have this list defined somewhere
    text_position = (np.average(np.array(cities_locations)[:, 0]), HEIGHT - 1.5 * font_size)

    screen.blit(text_surface, text_position)

In [ ]:
# FUNÇÕES DO PROFESSOR
# draw_functions.py

# -*- coding: utf-8 -*-
"""
Created on Wed Dec 27 13:33:42 2023

@author: SérgioPolimante
"""

## problem source: https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html

att_48_cities_locations = [(6734, 1453),
(2233 , 10),
(5530, 1424),
 (401, 841),
(3082, 1644),
(7608, 4458),
(7573, 3716),
(7265, 1268),
(6898, 1885),
(1112, 2049),
(5468, 2606),
(5989, 2873),
(4706, 2674),
(4612, 2035),
(6347, 2683),
(6107, 669),
(7611, 5184),
(7462, 3590),
(7732, 4723),
(5900, 3561),
(4483, 3369),
(6101, 1110),
(5199, 2182),
(1633, 2809),
(4307, 2322),
 (675, 1006),
(7555, 4819),
(7541, 3981),
(3177, 756),
(7352, 4506),
(7545, 2801),
(3245, 3305),
(6426, 3173),
(4608, 1198),
 (23, 2216),
(7248, 3779),
(7762, 4595),
(7392, 2244),
(3484, 2829),
(6271, 2135),
(4985, 140),
(1916, 1569),
(7280, 4899),
(7509, 3239),
 (10, 2676),
(6807, 2993),
(5185, 3258),
(3023, 1942)]

att_48_cities_order = [1,
8,
38,
31,
44,
18,
7,
28,
6,
37,
19,
27,
17,
43,
30,
36,
46,
33,
20,
47,
21,
32,
39,
48,
5,
42,
24,
10,
45,
35,
4,
26,
2,
29,
34,
41,
16,
22,
3,
23,
14,
25,
13,
11,
12,
15,
40,
9,
1,]

# CÓDIGO

In [ ]:
# define constant values
# pygame
WIDTH, HEIGHT = 800, 400
NODE_RADIUS = 10
FPS = 30
PLOT_X_OFFSET = 450

# GA
N_CITIES = 15
POPULATION_SIZE = 100
N_GENERATIONS = None
MUTATION_PROBABILITY = 0.5

# define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

In [ ]:
# initialize problem
# using random cities generation
cities_locations = [(random.randint(NODE_RADIUS + PLOT_X_OFFSET, WIDTH - NODE_RADIUS), random.randint(NODE_RADIUS, HEIGHT - NODE_RADIUS)) for _ in range(N_CITIES)]

# # Using Deault Problems: 10, 12 or 15
# WIDTH, HEIGHT = 800, 400
# cities_locations = default_problems[15]


# Using att48 benchmark
# WIDTH, HEIGHT = 1500, 800
# att_cities_locations = np.array(att_48_cities_locations)
# max_x = max(point[0] for point in att_cities_locations)
# max_y = max(point[1] for point in att_cities_locations)
# scale_x = (WIDTH - PLOT_X_OFFSET - NODE_RADIUS) / max_x
# scale_y = HEIGHT / max_y
# cities_locations = [(int(point[0] * scale_x + PLOT_X_OFFSET),
#                      int(point[1] * scale_y)) for point in att_cities_locations]
# target_solution = [cities_locations[i-1] for i in att_48_cities_order]
# fitness_target_solution = calculate_fitness(target_solution)
# print(f"Best Solution: {fitness_target_solution}")
# ----- Using att48 benchmark

In [ ]:
# initialize pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TSP Solver using Pygame")
clock = pygame.time.Clock()
generation_counter = itertools.count(start=1) # start the counter at 1

In [ ]:
# create initla population
# TODO:- use some heuristic like Nearest Neighbour our Convex Hull to initialize
population = generate_random_population(cities_locations, POPULATION_SIZE)
best_fitness_values = []
best_solutions = []

In [22]:
# main game loop
running = True
while running:
  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      running = False
    elif event.type == pygame.KEYDOWN:
      if event.type == pygame.K_q:
        running = False

  generation = next(generation_counter)

  screen.fill(WHITE)

  population_fitness = [calculate_fitness(
      individual) for individual in population]

  population, population_fitness = sort_population(population, population_fitness)

  best_fitness = calculate_fitness(population[0])
  best_solution = population[0]

  best_fitness_values.append(best_fitness)
  best_solutions.append(best_solution)

  draw_plot(screen, list(range(len(best_fitness_values))),
            best_fitness_values, y_label="Fitness - Distance (pxls)")

  draw_cities(screen, cities_locations, RED, NODE_RADIUS)
  draw_paths(screen, best_solution, BLUE, width=3)
  draw_paths(screen, population[1], rgb_color=(128, 128, 128), width=1)

  print(f"Generation {generation}: Best fitness = {round(best_fitness, 2)}")

  new_population = [population[0]] # Keep the best individual: ELITISM

  while len(new_population) < POPULATION_SIZE:

    # selection
    # simple selection based on first 10 best solutions
    # parent1, parent2 = random.choices(population[:10], k=2)

    # solution based on fitness probability
    probability = 1 / np.array(population_fitness)
    parent1, parent2 = random.choices(population, weights=probability, k=2)

    # child1 = order_crossover(parent1, parent2)
    child1 = order_crossover(parent1, parent1)

    child1 = mutate(child1, MUTATION_PROBABILITY)

    new_population.append(child1)

  population = new_population

  pygame.display.flip()
  clock.tick(FPS)

# TODO: save the best individual in a file if it is better than the one saved.

# exit software
pygame.quit()
sys.exit()

Generation 248: Best fitness = 1568.11
Generation 249: Best fitness = 1568.11
Generation 250: Best fitness = 1568.11
Generation 251: Best fitness = 1568.11
Generation 252: Best fitness = 1568.11
Generation 253: Best fitness = 1568.11
Generation 254: Best fitness = 1568.11
Generation 255: Best fitness = 1568.11
Generation 256: Best fitness = 1568.11
Generation 257: Best fitness = 1568.11
Generation 258: Best fitness = 1568.11
Generation 259: Best fitness = 1568.11
Generation 260: Best fitness = 1568.11
Generation 261: Best fitness = 1568.11
Generation 262: Best fitness = 1568.11
Generation 263: Best fitness = 1568.11
Generation 264: Best fitness = 1568.11
Generation 265: Best fitness = 1568.11
Generation 266: Best fitness = 1568.11
Generation 267: Best fitness = 1568.11
Generation 268: Best fitness = 1568.11
Generation 269: Best fitness = 1568.11
Generation 270: Best fitness = 1568.11
Generation 271: Best fitness = 1568.11
Generation 272: Best fitness = 1568.11
Generation 273: Best fitn

KeyboardInterrupt: 